# Processing the data: from df_complete to useful and targeted data

In [ ]:
# Libraries to import
import pandas as pd
import numpy as np
import matplotlib as matplot
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import pickle
from scipy.stats import ttest_ind

#interactive plots
import plotly.express as px
import plotly as py
import plotly.graph_objects as go

# Networkx
import pyvis
import networkx as nx
from pyvis import network
from pyvis.network import Network
import matplotlib.image as mpimg

# From M3_df_YEAR_complete.pkl to origine_media_YEAR.pickel

In [ ]:
DATA_PATH = './data/'
DATA_PATH2020 = DATA_PATH + 'data_2020/'
FILE2020 = DATA_PATH2020 + 'M3_df_2020_final.pkl'


DATA_PATH = './data/'

DATA_PATH2015 = DATA_PATH + 'data2015/'
DATA_PATH2016 = DATA_PATH + 'data2016/'
DATA_PATH2017 = DATA_PATH + 'data2017/'
DATA_PATH2018 = DATA_PATH + 'data2018/'
DATA_PATH2019 = DATA_PATH + 'data2019/'
DATA_PATH2020 = DATA_PATH + 'data2020/'

FILE2015 = DATA_PATH2015 + 'M3_df_2015_complete.pkl'
FILE2016 = DATA_PATH2016 + 'M3_df_2016_complete.pkl'
FILE2017 = DATA_PATH2017 + 'M3_df_2017_complete.pkl'
FILE2018 = DATA_PATH2018 + 'M3_df_2018_complete.pkl'
FILE2019 = DATA_PATH2019 + 'M3_df_2019_complete.pkl'
FILE2020 = DATA_PATH2020 + 'M3_df_2020_complete.pkl'

FILES = [FILE2015, FILE2016, FILE2017, FILE2018, FILE2019, FILE2020]

In [ ]:
with open(FILE2015, 'rb') as input_file:
    df_2015_base = pickle.load(input_file)
df_2015 = df_2015_base.groupby(['gender', 'age_range', 'quotation_length', 'media_country'])
df_2015['year'] = '2015'
df_2015.to_pickle('data/data2015/origine_media_2015.pickle')

In [ ]:
with open(FILE2016, 'rb') as input_file:
    df_2016_base = pickle.load(input_file)
df_2016 = df_2016_base.groupby(['gender', 'age_range', 'quotation_length', 'media_country'])
df_2016['year'] = '2016'
df_2016.to_pickle('data/data2016/origine_media_2016.pickle')

In [ ]:
with open(FILE2017, 'rb') as input_file:
    df_2017_base = pickle.load(input_file)
df_2017 = df_201_base7.groupby(['gender', 'age_range', 'quotation_length', 'media_country'])
df_2017['year'] = '2017'
df_2017.to_pickle('data/data2017/origine_media_2017.pickle')

In [ ]:
with open(FILE2018, 'rb') as input_file:
    df_2018_base = pickle.load(input_file)
df_2018 = df_2018_base.groupby(['gender', 'age_range', 'quotation_length', 'media_country'])
df_2018['year'] = '2018'
df_2018.to_pickle('data/data2018/origine_media_2018.pickle')

In [ ]:
with open(FILE2019, 'rb') as input_file:
    df_2019_base = pickle.load(input_file)
df_2019 = df_2019_base.groupby(['gender', 'age_range', 'quotation_length', 'media_country'])
df_2019['year'] = '2019'
df_2019.to_pickle('data/data2019/origine_media_2019.pickle')

In [ ]:
with open(FILE2020, 'rb') as input_file:
    df_2020_base = pickle.load(input_file)
df_exploded = df_2020_base.explode('media_country')
df_2020 = df_exploded.groupby(['media_country','gender', 'age_range', 'quotation_length']).quoteID.count().to_frame(name='count').sort_values(["media_country","count"],ascending=False).reset_index()
df_2020['year'] = '2020'
df_2020.to_pickle('data/data2020/origine_media_2020.pickle')

## From M3_df_complete to M3_df_media_country_groupby_proportion.pkl

In [ ]:
def create_df_media_country(df):
    df_media_country_explode = df.explode("media_country")
    df_media_country = df_media_country_explode.groupby(["media_country","gender"]).quoteID.count().to_frame(name="count").sort_values(['count'],ascending=False).reset_index()
    df_total_per_media_country = df_media_country.groupby(['media_country'])["count"].sum().to_frame(name="Total").reset_index()
    df_media_country = df_media_country.merge(df_total_per_media_country, on='media_country')
    df_media_country['Proportion']  = (df_media_country['count']/df_media_country['Total'])*100
    return df_media_country

In [ ]:
df_2015_media_country = create_df_media_country(df_2015_base) # df_2015_base is M3_df_complete_2025 and has been downloaded above (control not modified during process !)
df_2016_media_country = create_df_media_country(df_2016_base)
df_2017_media_country = create_df_media_country(df_2017_base)
df_2018_media_country = create_df_media_country(df_2018_base)
df_2019_media_country = create_df_media_country(df_2019_base)
df_2020_media_country = create_df_media_country(df_2020_base)

In [ ]:
# Save df_YEAR_media_country as pickle that we will reload afterwards
with open(DATA_PATHYEAR + 'M3_df_YEAR_media_country_groupby_proportion.pkl', 'wb') as output:
    pickle.dump(df_YEAR_media_country, output)

In [ ]:
### Load each dataframe groupedby
with open(DATA_PATHYEAR + 'M3_df_YEAR_media_country_groupby_proportion.pkl', 'rb') as input_file:
    df_YEAR_media_country = pickle.load(input_file)

In [ ]:
### Merge for each media_country
df_media_country1 = pd.merge(df_2015_media_country, df_2016_media_country, on=["media_country","gender"], 
                             how='outer', suffixes = ('_2015', '_2016'))
df_2017_media_country = df_2017_media_country.add_suffix('_2017').rename({"media_country_2017":"media_country", "gender_2017":"gender"},axis=1)
df_media_country2 = pd.merge(df_media_country1, df_2017_media_country, on=["media_country","gender"], how='outer')
df_2018_media_country = df_2018_media_country.add_suffix('_2018').rename({"media_country_2018":"media_country", "gender_2018":"gender"},axis=1)
df_media_country3 = pd.merge(df_media_country2, df_2018_media_country, on=["media_country","gender"], how='outer')
df_2019_media_country = df_2019_media_country.add_suffix('_2019').rename({"media_country_2019":"media_country", "gender_2019":"gender"},axis=1)
df_media_country4 = pd.merge(df_media_country3, df_2019_media_country, on=["media_country","gender"], how='outer')
df_2020_media_country = df_2020_media_country.add_suffix('_2020').rename({"media_country_2020":"media_country", "gender_2020":"gender"},axis=1)
df_media_country = pd.merge(df_media_country4, df_2020_media_country, on=["media_country","gender"], how='outer')

In [ ]:
# Save it !
with open(DATA_PATH + 'M3_df_media_country_groupby_proportion.pkl', 'wb') as output:
    pickle.dump(df_media_country, output)

It's on the drive in the folder: 'DataFrame_female_proportion_all_media_country' in M3 :)

## Few lines for the plot! (too lazy to create a new lab) just copy paste it at the begining with the plot 'mean percentage where we selected the 3 country UK USA SA 

In [ ]:
with open(DATA_PATH + 'M3_df_media_country_groupby_proportion.pkl', 'rb') as input_file:
    df_media_country = pickle.load(input_file)

In [ ]:
df_media_country.sort_values(["media_country","gender"], inplace=True)
df_media_country.set_index(["media_country","gender"], inplace=True)
df_media_country_tr = df_media_country.transpose().filter(regex='Proportion', axis=0)

In [ ]:
rows, nber_countries_x2 = df_media_country_tr.shape
fig, axes = plt.subplots(nrows=7, ncols=3, figsize=(40,55), sharey=True)
#fig.set_size_inches(20, 20)
list_axes = axes.flatten()
country=0
plot=0
while country < (nber_countries_x2):
    to_plot = df_media_country_tr.iloc[:, country:country+2].filter(regex='Female', axis=1)
    name = to_plot.columns[0][0]
    ax = list_axes[plot]

    to_plot.plot.bar(ax=ax, color=[colors[0], colors[1]])
    ax.set_title(str(name), size=25)
    ax.tick_params(axis='both', which='major', labelsize=15)
    
    if country % 3 == 0:
        ax.set_ylabel('Count', size=16)
        
    country += 2
    plot += 1

#plt.suptitle('Distributions of numerical columns', size=30)
plt.tight_layout(pad=5)
plt.show() 
#fig.savefig('Proportion_quotes_media_country_female.pdf')

# From M3_df_YEAR_complete.pkl to UK_USA_SA_YEAR_clean.pkl

In [ ]:
DATA_PATH = './data/'
DATA_PATH2020 = DATA_PATH + 'data_2020/'
FILE2020 = DATA_PATH2020 + 'M3_df_2020_final.pkl'

DATA_PATH = './data/'

DATA_PATH2015 = DATA_PATH + 'data2015/'
DATA_PATH2016 = DATA_PATH + 'data2016/'
DATA_PATH2017 = DATA_PATH + 'data2017/'
DATA_PATH2018 = DATA_PATH + 'data2018/'
DATA_PATH2019 = DATA_PATH + 'data2019/'
DATA_PATH2020 = DATA_PATH + 'data2020/'

FILE2015 = DATA_PATH2015 + 'M3_df_2015_complete.pkl'
FILE2016 = DATA_PATH2016 + 'M3_df_2016_complete.pkl'
FILE2017 = DATA_PATH2017 + 'M3_df_2017_complete.pkl'
FILE2018 = DATA_PATH2018 + 'M3_df_2018_complete.pkl'
FILE2019 = DATA_PATH2019 + 'M3_df_2019_complete.pkl'
FILE2020 = DATA_PATH2020 + 'M3_df_2020_complete.pkl'

FILES = [FILE2015, FILE2016, FILE2017, FILE2018, FILE2019, FILE2020]

## Same process for all years: YEAR in [2015, 2016, 2017, 2018, 2019, 2020]

In [ ]:
#Open FILE
with open(FILE2020, 'rb') as input_file:
    df_2020 = pickle.load(input_file)

In [ ]:
#Keep only line where 'sitename' and 'media_country' lists had the same length (otherwise not explodable and concerned only negligible lines)
df_sitenames_groupby = df_2020['sitenames'].apply(lambda x: len(x)).to_frame(name='count_sitenames')
df_media_country_groupby = df_2020['media_country'].apply(lambda x: len(x)).to_frame(name='count_country')
df_comparison = pd.concat([df_2020['quoteID'], df_sitenames_groupby, df_media_country_groupby], axis=1).reset_index(drop=True)
keep_quoteIDs = df_comparison['quoteID'][df_comparison['count_sitenames']==df_comparison['count_country']]
keep_quoteIDs = keep_quoteIDs.to_frame(name='quoteID_keep')
df_length_equal = df_2020[df_2020['quoteID'].isin(keep_quoteIDs['quoteID_keep'])]

In [ ]:
#Explode to have row with 1 single element instead of list
df_exploded = df_length_equal.explode(['media_country','sitenames','tags'])
df_exploded = df_exploded.explode('tags') #once more for tags because still as an array

In [ ]:
#Keep only countries of interest and drop columns which we are not considering in the final analysis
df_UK_USA_SA_YEAR = df_exploded[(df_exploded['media_country']=='United Kingdom') 
                                                   | (df_exploded['media_country']== 'United States of America') 
                                                   | (df_exploded['media_country']=='South Africa')]

df_UK_USA_SA_YEAR.drop(['qids','date','urls','domain','Description','date_of_birth','media_country_qid','numOcurrences','Continent'],axis=1,inplace=True)
df_UK_USA_SA_YEAR.reset_index(drop=True, inplace = True)
#df_UK_USA_SA_YEAR.to_pickle('data/dataYEAR/df_UK_USA_SA_clean_YEAR.pkl')

In [ ]:
df_UK_USA_SA_YEAR